# Instalo e importo librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=c62c23cdf2eebdffbf1fb8c19c1409d2d54a9ae537518aca0a098eb82aee1570
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Creo sesión de spark y leo csvs

In [ ]:
downloaded1 = drive.CreateFile({'id':"1gDD0Mn9bYoj4RQlPb7V2_3mJ_-Ikmit8"})
downloaded1.GetContentFile('GooglePlayStore.csv')
downloaded2 = drive.CreateFile({'id':"10xgOf2rORcGlcKPME3cHGHssXGzNemer"})
downloaded2.GetContentFile('GooglePlayStore_User_Reviews.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df1 = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('GooglePlayStore.csv', header=True, inferSchema=True)
df2 = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('GooglePlayStore_User_Reviews.csv', header=True, inferSchema=True)
apps_details = df1.rdd
user_reviews = df2.rdd

# Ejercicio 14

¿Cuál es la aplicación gratis con mayor ratio de reviews positivas? (⭐⭐)

Los valores de la columna "Reviews" de apps_details no coinciden en absoluto con la cantidad de reviews que tengo realmente en user_reviews.

Por ejemplo, si una app figura con 1000 reviews en el primer dataframe pero en el segundo solo tengo 40 reviews de esta, me falta un montón de información y calcular #(reviews positivas)/10000 sería erróneo ya que desconozco la información sobre la gran mayoría de la reviews de la app. Lo correcto sería calcular #(reviews positivas)/40.

Simplemente tendré en cuenta la cantidad de reviews por aplicación que tengo en user_reviews, ya que estas las conozco y sé si son negativas, neutrales o positivas.

Hay apps que están repetidas. La mayoría de las apps repetidas tiene el dato de su tamaño igual, pero desconozco si esto sucede en TODOS los casos, quizás haya discrepancias. Tomaré las que aparezcan como Free y eliminaré repetidas después (con el reduceByKey).

Antes, mapeo para quedarme las aplicaciones como clave y como valor un 1, que luego me servirá para contar la cantidad total de reviews.


In [ ]:
free_apps = apps_details.filter(lambda x: x.Type == 'Free').map(lambda x: (x.App,1)).reduceByKey(lambda a,b: b)

Filtro las reviews nan o None ya que las considero un error del archivo (no tienen sentiment ya que sin review no se puede saber esto, no me sirven para lo que quiero calcular).

Uso distinct para eliminar reviews que se encuentran repetidas (aparecen con todos los datos iguales y provoca que algunas apps terminen sumando más reviews positivas que las que realmente tienen).

Haré un map para quedarme las aplicaciones como clave y como valor True o False, según tengan sentimiento positivo o no.

In [ ]:
reviews_by_sentiment = user_reviews.filter(lambda x: (x.Translated_Review != 'nan') & (x.Translated_Review is not None)).distinct()\
.map(lambda x: (x.App, x.Sentiment == 'Positive'))

Hago un inner join entre los rdd ya que voy a necesitar datos de ambos para este ejercicio. ¿Por qué inner?

Si me quedo con una app que está en apps_details pero no en user_reviews, no tendré review para esta y no me servirá

Si tengo el caso opuesto, tendré una review para una app de la que desconozco si es gratis o paga, por lo que tampoco me servirá.

Hago el join entre los dos rdd que generé, de modo que me quedan las App como clave y como (True o False, 1). Hago un reduceByKey para sumar las reviews (True es 1 y False es 0, sumando obtengo la cantidad de positivas, los unos de la segunda posición se suman y obtengo la cantidad total). Mapeo para tener como valor el ratio de reviews positivas.

Cacheo lo anterior ya que tengo que hacer más de una acción sobre esto:

como no hay una única app de ratio máximo, daré el conjunto de ellas. Para esto, calculo cual es el ratio máximo y luego filtro usando este (puedo hacer collect ya que tengo un conjunto acotado de apps).



In [ ]:
apps_by_positive_reviews_ratio = reviews_by_sentiment.join(free_apps).reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))\
.map(lambda x: (x[0], x[1][0]/x[1][1])).cache()

In [ ]:
max_ratio = apps_by_positive_reviews_ratio.reduce(lambda x,y: x if x[1] > y[1] else y)[1]

In [ ]:
apps_by_positive_reviews_ratio.filter(lambda x: x[1] == max_ratio).collect()

[('Amazon Prime Video', 1.0),
 ('Baby Panda’s Juice Shop', 1.0),
 ('Best Fiends - Free Puzzle Game', 1.0),
 ('CM Flashlight (Compass, SOS)', 1.0),
 ('Calculator Plus Free', 1.0),
 ('Couple - Relationship App', 1.0),
 ('Daniel Tiger for Parents', 1.0),
 ('DashClock Widget', 1.0),
 ('Down Dog: Great Yoga Anywhere', 1.0),
 ('Drawing for Kids Learning Games for Toddlers age 3', 1.0),
 ('FlipaClip - Cartoon animation', 1.0),
 ('Google Primer', 1.0),
 ('Google Translate', 1.0),
 ('7 Day Food Journal Challenge', 1.0),
 ('Home workouts - fat burning, abs, legs, arms,chest', 1.0),
 ('HomeWork', 1.0),
 ('All-in-One Mahjong 3 FREE', 1.0),
 ('Apartment Decorating Ideas', 1.0),
 ('Calendar+ Schedule Planner App', 1.0),
 ('CallApp: Caller ID, Blocker & Phone Call Recorder', 1.0),
 ('Google Slides', 1.0),
 ('Google Trips - Travel Planner', 1.0),
 ('3D Live Neon Weed Launcher', 1.0),
 ('Bed Time Fan - White Noise Sleep Sounds', 1.0),
 ('Best Ovulation Tracker Fertility Calendar App Glow', 1.0),
 ('Bri

Calculé el max_ratio ya que no puedo adivinar cuál va a ser (es 1 debido a que hay varias apps con sólo reviews positivas, pero no creo que sea correcto usar 1 directamente, estaría asumiendo algo que puede no ser cierto).